In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

1、获取股票列表

In [2]:
df = pd.read_excel('./股票列表.xlsx', engine='openpyxl')
code_list = []
# 遍历第二列中的股票代码
for stock_code in df['代码']:
    old_code = stock_code
    if stock_code.startswith('SH'):
        stock_code = f"{stock_code[2:]}.{'SS'}"
    elif stock_code.startswith('SZ'):
        stock_code = f"{stock_code[2:]}.{'SZ'}"
    code_list.append(stock_code)
code_list

['300905.SZ',
 '300561.SZ',
 '300287.SZ',
 '300125.SZ',
 '300086.SZ',
 '300386.SZ',
 '000566.SZ',
 '000850.SZ',
 '002036.SZ',
 '603123.SS',
 '600676.SS',
 '600841.SS',
 '601162.SS',
 '600148.SS',
 '600650.SS',
 '600635.SS',
 '605218.SS',
 '600847.SS',
 '000062.SZ',
 '603883.SS',
 '002104.SZ',
 '001298.SZ',
 '000750.SZ',
 '000678.SZ',
 '000627.SZ',
 '603306.SS',
 '600838.SS',
 '600611.SS',
 '603626.SS',
 '002823.SZ',
 '600826.SS',
 '002640.SZ',
 '002520.SZ',
 '600828.SS',
 '002717.SZ',
 '300736.SZ',
 '301607.SZ',
 '002456.SZ',
 '300641.SZ',
 '600418.SS',
 '300209.SZ',
 '300758.SZ',
 '605289.SS',
 '001696.SZ',
 '300077.SZ',
 '300464.SZ',
 '600686.SS',
 '002388.SZ',
 '601890.SS',
 '601086.SS',
 '300013.SZ',
 '605598.SS',
 '002827.SZ',
 '600769.SS',
 '002208.SZ',
 '600576.SS',
 '300518.SZ',
 '300619.SZ',
 '300436.SZ',
 '603036.SS',
 '300068.SZ',
 '002312.SZ',
 '002528.SZ',
 '600898.SS',
 '000536.SZ',
 '000712.SZ',
 '600358.SS',
 '000908.SZ',
 '000584.SZ',
 '603559.SS',
 '605228.SS',
 '6008

2、定义起始日期

In [3]:
start_date = '2015-01-01'
end_date = '2024-08-31'

3、获取所有股票的数据整合到一个文件中

In [4]:
df = pd.read_csv('./all_data.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume,code
0,2020-11-05,55.000000,62.150002,51.000000,51.029999,47.681606,21472552,300905.SZ
1,2020-11-06,47.005001,51.520000,46.505001,48.599998,45.411053,13282442,300905.SZ
2,2020-11-09,48.500000,49.845001,47.119999,48.740002,45.541874,11263252,300905.SZ
3,2020-11-10,48.000000,49.439999,47.000000,47.250000,44.149632,8399338,300905.SZ
4,2020-11-11,46.900002,49.055000,46.595001,47.700001,44.570107,8407878,300905.SZ
...,...,...,...,...,...,...,...,...
20735,2024-08-26,5.680000,5.840000,5.650000,5.820000,5.820000,4624700,603123.SS
20736,2024-08-27,5.830000,5.890000,5.610000,5.660000,5.660000,4861100,603123.SS
20737,2024-08-28,5.680000,5.800000,5.550000,5.700000,5.700000,4983901,603123.SS
20738,2024-08-29,5.680000,5.770000,5.640000,5.760000,5.760000,4388100,603123.SS


4、数据清洗

In [5]:
# 过滤掉code列中以'300'开头的行
df = df[~df['code'].str.startswith('300')]
df

,Date,Open,High,Low,Close,Adj Close,Volume,code
11834,2015-01-05,7.440,7.490,7.290,7.385,7.192119,12799906,000566.SZ
11835,2015-01-06,7.390,7.710,7.365,7.595,7.396634,14522622,000566.SZ
11836,2015-01-07,7.645,8.050,7.645,7.755,7.552455,24060822,000566.SZ
11837,2015-01-08,7.800,7.885,7.605,7.705,7.503761,10945550,000566.SZ
11838,2015-01-09,7.845,8.090,7.845,7.945,7.737493,25795612,000566.SZ
...,...,...,...,...,...,...,...,...
20735,2024-08-26,5.680,5.840,5.650,5.820,5.820000,4624700,603123.SS
20736,2024-08-27,5.830,5.890,5.610,5.660,5.660000,4861100,603123.SS
20737,2024-08-28,5.680,5.800,5.550,5.700,5.700000,4983901,603123.SS
20738,2024-08-29,5.680,5.770,5.640,5.760,5.760000,4388100,603123.SS


In [6]:
df.set_index('code', inplace=True)

In [7]:
# 计算每只股票的当日涨幅
df['Pct_Chg'] = df.groupby(level=0)['Close'].pct_change()
# 将涨幅数据转换为百分比形式
df['Pct_Chg'] *= 100
# 去除Pct_Chg为空的值
df = df.dropna(subset=['Pct_Chg'])
df

D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\envs\strategy\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg
code,,,,,,,,
000566.SZ,2015-01-06,7.390,7.710,7.365,7.595,7.396634,14522622,2.843596
000566.SZ,2015-01-07,7.645,8.050,7.645,7.755,7.552455,24060822,2.106653
000566.SZ,2015-01-08,7.800,7.885,7.605,7.705,7.503761,10945550,-0.644748
000566.SZ,2015-01-09,7.845,8.090,7.845,7.945,7.737493,25795612,3.114864
000566.SZ,2015-01-12,7.945,8.100,7.730,8.065,7.854358,17574292,1.510376
...,...,...,...,...,...,...,...,...
603123.SS,2024-08-26,5.680,5.840,5.650,5.820,5.820000,4624700,1.748258
603123.SS,2024-08-27,5.830,5.890,5.610,5.660,5.660000,4861100,-2.749146
603123.SS,2024-08-28,5.680,5.800,5.550,5.700,5.700000,4983901,0.706713


5、和上证指数数据合并

In [8]:
df_szzs = pd.read_csv('000001.csv')
df_szzs.set_index('code', inplace=True)
# 计算每只股票的当日涨幅
df_szzs['Pct_Chg_szzs'] = df_szzs.groupby(level=0)['Close'].pct_change()
# 将涨幅数据转换为百分比形式
df_szzs['Pct_Chg_szzs'] *= 100
# 去除Pct_Chg为空的值
df_szzs = df_szzs.dropna(subset=['Pct_Chg_szzs'])

df_szzs

,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg_szzs
code,,,,,,,,
000001.SS,2015-01-06,3330.799072,3394.224121,3303.184082,3351.446045,3351.446045,501700,0.027667
000001.SS,2015-01-07,3326.648926,3374.895996,3312.210938,3373.954102,3373.954102,391900,0.671592
000001.SS,2015-01-08,3371.957031,3381.565918,3285.094971,3293.456055,3293.456055,371100,-2.385867
000001.SS,2015-01-09,3276.965088,3404.833984,3267.509033,3285.412109,3285.412109,410200,-0.244240
000001.SS,2015-01-12,3258.212891,3275.185059,3191.582031,3229.315918,3229.315918,322100,-1.707432
...,...,...,...,...,...,...,...,...
000001.SS,2024-08-26,2855.474121,2860.655029,2844.383057,2855.520020,2855.520020,227400,0.040363
000001.SS,2024-08-27,2850.876953,2853.780029,2843.157959,2848.730957,2848.730957,230700,-0.237752
000001.SS,2024-08-28,2844.326904,2852.562012,2830.961914,2837.428955,2837.428955,228700,-0.396738


In [9]:
# 重置索引，以便合并
df = df.reset_index()
df_szzs = df_szzs.reset_index()
# 合并两个DataFrame
# 这里我们按照'Date'列进行合并，并且使用左连接，这样只会保留df中的日期
df = df.merge(df_szzs[['Date', 'Pct_Chg_szzs']], on='Date', how='left')
df

,code,Date,Open,High,Low,Close,Adj Close,Volume,Pct_Chg,Pct_Chg_szzs
0,000566.SZ,2015-01-06,7.390,7.710,7.365,7.595,7.396634,14522622,2.843596,0.027667
1,000566.SZ,2015-01-07,7.645,8.050,7.645,7.755,7.552455,24060822,2.106653,0.671592
2,000566.SZ,2015-01-08,7.800,7.885,7.605,7.705,7.503761,10945550,-0.644748,-2.385867
3,000566.SZ,2015-01-09,7.845,8.090,7.845,7.945,7.737493,25795612,3.114864,-0.244240
4,000566.SZ,2015-01-12,7.945,8.100,7.730,8.065,7.854358,17574292,1.510376,-1.707432
...,...,...,...,...,...,...,...,...,...,...
8897,603123.SS,2024-08-26,5.680,5.840,5.650,5.820,5.820000,4624700,1.748258,0.040363
8898,603123.SS,2024-08-27,5.830,5.890,5.610,5.660,5.660000,4861100,-2.749146,-0.237752
8899,603123.SS,2024-08-28,5.680,5.800,5.550,5.700,5.700000,4983901,0.706713,-0.396738
8900,603123.SS,2024-08-29,5.680,5.770,5.640,5.760,5.760000,4388100,1.052639,-0.504788
